<a href="https://colab.research.google.com/github/githbsingh/pyspark/blob/main/pySpark_Transformations_Set_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# File location and type
file_location = "/FileStore/tables/part_00000.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
df.head()

Out[3]: Row(_c0='1', _c1='2013-07-25 00:00:00.0', _c2='11599', _c3='CLOSED')

In [ ]:
print(type(df))
order = df.rdd.map(lambda row : ",".join(str(x) for x in row))
print(type(order))
for i in order.take(5) : print(i)

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.rdd.PipelinedRDD'>
1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE


#### Get the number of customers ordered in the month of Jul and Aug

In [ ]:
julOrd = order.filter(lambda x: (x.split(',')[1].split('-')[1])=='07').map(lambda x: x.split(',')[2])
for i in julOrd.take(7) : print(i)

11599
256
12111
8827
11318
7130
4530


In [ ]:
julOrd.count()

Out[17]: 6001

In [ ]:
augOrd = order.filter(lambda x: (x.split(',')[1].split('-')[1])=='08').map(lambda x: x.split(',')[2])
for i in augOrd.take(7) : print(i)

11607
5105
7802
553
1604
1695
7018


In [ ]:
augOrd.count()

Out[18]: 5680

In [ ]:
julAugOrd = julOrd.union(augOrd)
julAugOrd.count()

Out[19]: 11681

In [ ]:
julAugOrd.distinct().count()

Out[20]: 7633

#### Orders Applied in the month of July and Aug both

In [ ]:
julOrd.intersection(augOrd).count()


Out[22]: 1759

In [ ]:
rdd1 = sc.parallelize([1,2,3,3,3,3])
rdd2 = sc.parallelize([1,3,5])
rdd1.intersection(rdd2).collect()

Out[23]: [1, 3]

#### Subtract

In [ ]:
rdd1 = sc.parallelize([1,2,3,3,3,3])
rdd2 = sc.parallelize([1,3,5])
rdd1.subtract(rdd2).collect()

Out[24]: [2]

In [ ]:
rdd2.subtract(rdd1).collect()

Out[26]: [5]

In [ ]:
rdd = sc.parallelize(range(100), 4)

In [ ]:
for i in rdd.take(5) : print(i)

0
1
2
3
4


### Sample (Transformations operation)

In [ ]:
rdd.sample(withReplacement=False, fraction=0.1, seed=10).collect()

Out[34]: [5, 19, 20, 36, 37, 46, 47, 51, 65, 70, 72, 77, 96]

In [ ]:
rdd.sample(withReplacement=False, fraction=0.1, seed=9).collect()

Out[33]: [2, 21, 26, 40, 45, 47, 61, 62, 71, 72, 80, 94, 95]

#### takeSample (Actions operation)

In [ ]:
rdd.takeSample(withReplacement=True, num=10, seed=5)

Out[70]: [72, 98, 97, 10, 44, 76, 18, 44, 21, 95]

#### Repartitioning and coalasce

In [ ]:
# Dump data for testign.
df = spark.range(1000000)
print(df.head())
df = df.select(df.id, df.id*2, df.id*3)
df = df.union(df)
df = df.union(df)
df = df.union(df)
df = df.union(df)
df = df.union(df)
RDD = df.rdd.map(lambda x: str(x[0]) + ',' + str(x[1]) + ',' + str(x[2]))


Row(id=0)


In [ ]:
RDD.coalesce(1).saveAsTextFile('/Users/manoharkumar_singh@yahoo.com/test_data')